# in this notebook, we are actually going to write the true HDF5 files we need.

In [1]:
import pandas as pd
import numpy as np
import h5py
from utils import get_photopath
from tqdm import tqdm
import os
from einops import rearrange

In [2]:
#mask = [os.path.exists(get_photopath(i)) for i in range(len(DF))]

In [3]:
#np.save('/rcfs/projects/mantis_shrimp/mantis_shrimp/data/exists_mask.npy',np.array(mask),)

In [4]:
mask = np.load('/rcfs/projects/mantis_shrimp/mantis_shrimp/data/exists_mask.npy')

In [5]:
csv_path='/rcfs/projects/mantis_shrimp/mantis_shrimp/data/spectroscopy/redshifts_withextinction.pkl'
csv_path2='/rcfs/projects/mantis_shrimp/mantis_shrimp/data/redshifts_broken_beck/SDSS_MGS/MGS_qwsaz123.csv'
csv_path3='/rcfs/projects/mantis_shrimp/mantis_shrimp/data/redshifts_broken_beck/WISE_PS1_STRM.csv'

DF = pd.read_pickle(csv_path,)

#next load SDSS_MGS csv, merge into
DF_pas = pd.read_csv(csv_path2,usecols=['bestObjID','zphot'])
DF_pas.drop_duplicates('bestObjID',inplace=True)
DF = pd.merge(DF,DF_pas,'left',left_on='photoObjID_survey',right_on='bestObjID')
DF = DF.drop('bestObjID',1)

#now load WISE_PS1_STRM csv, merge into
DF_wps = pd.read_csv(csv_path3,
                     usecols=['dstArcSec',
                              'cellDistance_Photoz',
                              'z_phot0',
                              'z_photErr',
                              'prob_Galaxy',
                              'photoObjID_survey'])
DF = pd.merge(DF,DF_wps,how='left',on='photoObjID_survey')

/tmp/ipykernel_164600/3335362331.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  DF = DF.drop('bestObjID',1)


In [6]:
#43 different hdf5 files. 
feature_size = 2*(32*32*2) + (5*170*170)

In [7]:
def load_image_data(indices):
    image_vectors = []
    for j in tqdm(indices):
        filename = get_photopath(j)
        if not(mask[j]): #file doesnt exist
            #we can do this now and then target these vectors later to modify.
            image_vector = np.zeros(feature_size).astype(np.float32)
            #Since hdf5 has compression, should maybe not cost ttoo much.
        else: #file does exist
            img = np.load(get_photopath(j),allow_pickle=True).item()

            unwise = rearrange(img['unwise'],'f h w -> (f h w)')
            galex = rearrange(img['galex'],'f h w -> (f h w)')
            panstarrs = rearrange(img['panstarrs'],'f h w -> (f h w)')
        
            image_vector = np.concatenate([galex,panstarrs,unwise])
            
        image_vectors.append(image_vector)
    #now we can place this into our hdf5 file
    image_vectors = np.stack(image_vectors)
    return image_vectors

In [24]:
# #Because I use a seed, this should allow me to always re-generate the mapping.
# all_indices = np.arange(len(mask))

# np.random.seed(2212024)
# random.seed(2212024)

# import random
# random.shuffle(all_indices)

# train_indices = all_indices[0:int(0.7*len(all_indices))]
# val_indices = all_indices[int(0.7*len(all_indices)):int(0.8*len(all_indices))]
# test_indices = all_indices[int(0.8*len(all_indices))::]

# #now sort into 16 vectors of equal length.
# train_indices = train_indices[0:16 * (len(train_indices)//16)].reshape(16,-1)
# val_indices = val_indices[0:16* (len(val_indices)//16)].reshape(16,-1)
# test_indices = test_indices[0:16 * (len(test_indices)//16)].reshape(16,-1)

# np.save('/rcfs/projects/mantis_shrimp/mantis_shrimp/data/npy_blocks/train_indices.npy',train_indices,)
# np.save('/rcfs/projects/mantis_shrimp/mantis_shrimp/data/npy_blocks/val_indices.npy',val_indices,)
# np.save('/rcfs/projects/mantis_shrimp/mantis_shrimp/data/npy_blocks/test_indices.npy',test_indices,)

In [8]:
train_indices = np.load('/rcfs/projects/mantis_shrimp/mantis_shrimp/data/npy_blocks/train_indices.npy',)
val_indices = np.load('/rcfs/projects/mantis_shrimp/mantis_shrimp/data/npy_blocks/val_indices.npy',)
test_indices = np.load('/rcfs/projects/mantis_shrimp/mantis_shrimp/data/npy_blocks/test_indices.npy',)

In [9]:
for i in range(0,16):
    filename = f'/rcfs/projects/mantis_shrimp/mantis_shrimp/data/npy_blocks/train/mantis_shrimp_{i}.npy'
    
    if os.path.exists(filename):
        continue
        
    #f.create_group('photometry')
    data = load_image_data(train_indices[i]) #return an image of size 100_000 x feature_size

    np.save(filename,data)
    #My other dataset already shows how to extract the remainder of the metadata I need. 
    #I'm going to use that instead, and save train_indices, val_indices, and test_indices so that I can access them
    #at any time.

 32%|██████████████████████▌                                                | 60217/189137 [5:28:38<11:43:35,  3.05it/s]


KeyboardInterrupt: 